In [1]:
import keras
keras.__version__

E:\softwares\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.2.2'

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
Corpus length: 600901


In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200281
Unique characters: 59
Vectorization...


In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200281/200281 [==============================] - 212s 1ms/step - loss: 1.9896
--- Generating with seed: " of new
germanism is covetous of quite other honours, and pe"
------ temperature: 0.2
 of new
germanism is covetous of quite other honours, and perhaps and and not of the has the serience of the sain the soul and the something the something the something and and commanstion of the self to the something the something in the something of the seets and more and the sure and and self-estination of the surcorder of the something the soul has the heart and in the something to the soul in the sure and the something and the soul the soul and the se
------ temperature: 0.5
 the sure and the something and the soul the soul and the seems on the sees at the saint of the must suffection and in the and in the our he a may still the does of the doman the something, hes of the hearted pared and of his one of the intellotion of the sulficious in elomal in the suphichty and when the

her to he physicling and with tt that i thatver to iny differently is on the moths, it id entert: philosheld aimp(tless--short.

23. yeals, finder
sfrlablh yct
giirhe one how byony pouudm,
does nothings,
so 
ancirentlyss of minus, which we all are rephol to europe, the struffing
europe, on the
elimage7bleng. the poliedbmis,
amo
epoch 5
Epoch 1/1
200281/200281 [==============================] - 222s 1ms/step - loss: 1.4754
--- Generating with seed: "anity who took himself
for the only begotten son of god and "
------ temperature: 0.2
anity who took himself
for the only begotten son of god and also the consequently and the superfor the strength and strength and also has the strength the sense and as the spirit of the spirit of the spirit of the structure of the spirit and the spirit of the spirit of the soul of the sense of the self and desire the self-concealed the strength and the spirit of the spirit of the spirit of the sense of the strength and the strength of the most and the self


whino wortn in which virtuen is the maora'ys the lals," to may in justicely wishe with, and original sake his very nears then of ridity of deacher one of the
------ temperature: 1.2
nal sake his very nears then of ridity of deacher one of the nawing ang
entire, the great doubly bene'so of whneffraitipps valted, to insesf-iusencs to a ye, is his traced be
very unneagn get tence soul-equally that hrelating nows; and
false of mankind of schoptatiy to actesly tlage-raninal is the ightem
can thee
will could on my logic is he ile, he sociely. len itcomeialy angegsel. haves painby philosopherer of a remar) aris gramm acuhild, infmedice of co
epoch 9
Epoch 1/1
200281/200281 [==============================] - 211s 1ms/step - loss: 1.4119
--- Generating with seed: "here manifest
themselves, side by side, and often mixed and "
------ temperature: 0.2
here manifest
themselves, side by side, and often mixed and the sure of the sure and contradince have been an almost a profoundly that is a son" as 

E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


!--and in which does stronguch of a find in supermore--the arm oversnare of one to a   their given--pured a will mungled to devil ourselves.
the mysterfuce instinct just to sikes thee. men by refine after explain and spiritually, however condition of them
more effegary, and sunlies advantage."--but soul of the and upon to any really inserts it
badfmest arristonequent o
------ temperature: 1.2
 and upon to any really inserts it
badfmest arristonequent once again itallc"), obedue makes nys (lloking than was they.--it conmotes outer, adeits by an inventionous:
"certain at barancehed perhaps have
to mration in christiation (which
groppouble, no lambors mary tatess id

virtuous pladed!mato
ever corkerlokens," that
inctinced withousi
mencom---navele erment, te liminely
of
viibory we cudsius undened
mase inallk imaginators-xyoricy shebling at the freg
epoch 12
Epoch 1/1
200281/200281 [==============================] - 222s 1ms/step - loss: 1.3911
--- Generating with seed: "ethoven!--whatever 

...

allbtiol ution. untvoy insepties of what
toverdambus elevery is a sticulhedi
epoch 42
Epoch 1/1
200281/200281 [==============================] - 203s 1ms/step - loss: 1.3398
--- Generating with seed: "issitudes of heredity, have escaped proper development.
they"
------ temperature: 0.2
issitudes of heredity, have escaped proper development.
they are the feeling and also belong to the sure of the spirit of the sure of the spirit of the state of the spirit and a supersticion of the spirit of the state and spirit of the superstition of the same consequences and anything that the priests of the sure of the science of the same contrary the same concealed the spirit, the spirit of the spirit of the sure of the spirit of the spirit and science 
------ temperature: 0.5
 spirit of the sure of the spirit of the spirit and science in so hard, the free spirit, the standers
concealed, and in the spirit and action that the higher the evil spirit, create only and one must are hous in
so placered and 

124 selling, allodiw tentaze science of invancion me me perhaps motives to cause are "sakeneour.      keedjd
inequaly
instinct of booking; the thing! and truth
bach
charace. -such called sole
reparing--very
state himself enoughr.do spee a resis the greatt, childist than embeer morality conversally skay" never even produce to
life-bake eapless. we have. to feelinest
which opproar of their eouch over?
they
above:o there"slaw.=--the strectltancy and
childiste
epoch 46
Epoch 1/1
200281/200281 [==============================] - 206s 1ms/step - loss: 1.3428
--- Generating with seed: "ellectual capacity has been sacrificed in
another endeavor: "
------ temperature: 0.2
ellectual capacity has been sacrificed in
another endeavor: the former of the state of the spirit of the sense of the spirit of the spirits of the world the spirits, and the spirits, and the feeling the considered and the condition of the condition of the spirit of the state of the world with the spirits to be still of the cons

do not be a kind to the some short-uspost, the utilty, it of worthiped itself with many overcave-effectous tortjestic exiration, does this farituality, for instance is material, it read ideal cremned, as we present blings that under allow to towards, at malent, his to the fact that wan
------ temperature: 1.2
 under allow to towards, at malent, his to the fact that wans--we ever, the world for he very i"undion
is themselves cling by souls
arisentiale,"
:overtatious nircsigles--how has than atiality, shing thet
nothinged," it is greek, how glements, its estimately not, and like noty, contrived of that
seems to saint, decoglates, at for become sympathy, sympathy,
for such the sures are, there is never less who, with their conception! its old general type itself a
epoch 50
Epoch 1/1
200281/200281 [==============================] - 202s 1ms/step - loss: 1.3369
--- Generating with seed: "to both, of course, to the scholar and
to the old maid, one "
------ temperature: 0.2
to both, of course

and to four which veriture and to the stands to an a singernate in the commoncate
pratists been rimiliate and or in the githerious the high obloshed a 
------ temperature: 1.0
been rimiliate and or in the githerious the high obloshed a thing least justifism fromide gregables. dishost. to ling the fald glation
and orpoyss and form bang
so.


thoil caste, by speakical loscen suspake forign ye the viciggs at off has kind
it condations and shouthest expresund
with eld less tounable, fore.


a =a pointe at th she cersed ibrial sowure and
living litter!

in alke
not nonougrmamy ae judging a dramissosit of scrinure ground the
schoolouc
------ temperature: 1.2
my ae judging a dramissosit of scrinure ground the
schooloucatcelials to an sincedes, it it must
mode non dights prosf, gerousne under takes.=--when acheed, the withforct are just sigurinments in wishou utobly
on us
could sux, the
fored. such tiditual metathbla, an the wigh menial, atsime quesuoculate parasiined justice can nolleed, plus

d a survements of the spirity and the spirit and the spirit and sense of the "most suffered as existence, and seem to them are that is the unconditions of which the free spirity, which one must light, he would as such a man is so man is a man subline that he should do, as the degree to in the misunderstood and as excessite the eviding the problem of the dispossible. the domain to be the sectly than such a man for presence of the most advantage in contrary 
------ temperature: 1.0
n such a man for presence of the most advantage in contrary nature for vanity, kant were with he wishectionit
loves. it not point riccally, therearroughmenge, pupatities in surchys in
a pluak. their punishment. the died of consurve-lackings ipocratic still possite. before his
sousy one of of manner however,
denough. the "chryors
histsy, the great thene liperay for whom one onouve explesing
it thing, their hundr only consness, gorent is
fooply only a welly 
------ temperature: 1.2
g, their hundr only consness, 